In [30]:
import g4f

def get_word_family_etymology(word):
    engine = g4f.client.Client()

    completion = engine.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"""
    I will provide you a word and you should write all word family of it with very brief meanings and differences (if any exist).
    Please respond in JSON and don't write extra explanations. The word family list should be complete for the word.
    For example, for the request: shrink
    response: {{
        "word_family": [
            {{"word_form": "shrink", "part_of_speech": "verb", "definition": "to contract or become smaller."}},
            {{"word_form": "shrinkage", "part_of_speech": "noun", "definition": "the process or result of shrinking."}},
            {{"word_form": "shrinkable", "part_of_speech": "adjective", "definition": "capable of being shrunk."}}
        ],
        "etymology": "The word 'shrink' originates from Middle English 'shrinken,' from Old English 'scrincan,' meaning 'to contract or shrivel.'"
    }}
    Tell me about the word '{word}'.
"""}])

    return completion.choices[0].message.content


In [35]:
import g4f

import json
import os
from tqdm import tqdm
import time


def get_word_family_etymology(word):
    engine = g4f.client.Client()

    completion = engine.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"""
    I will provide you a word and you should write all word family of it with very brief meanings and differences (if any exist).
    Please respond in JSON and don't write extra explanations. The word family list should be complete for the word.
    For example, for the request: shrink
    response: {{
        "word_family": [
            {{"word_form": "shrink", "part_of_speech": "verb", "definition": "to contract or become smaller."}},
            {{"word_form": "shrinkage", "part_of_speech": "noun", "definition": "the process or result of shrinking."}},
            {{"word_form": "shrinkable", "part_of_speech": "adjective", "definition": "capable of being shrunk."}}
        ],
        "etymology": "The word 'shrink' originates from Middle English 'shrinken,' from Old English 'scrincan,' meaning 'to contract or shrivel.'"
    }}
    Tell me about the word '{word}'.
"""}])

    return completion.choices[0].message.content



def convert_str_to_json(value):
    """ Attempts to convert a string to JSON. Strips certain characters if initial parsing fails. """
    try:
        return json.loads(value)
    except json.JSONDecodeError:
        # Stripping potentially problematic characters and retrying
        cleaned_value = value.replace('```', '').replace('json', '').replace('JSON', '')
        try:
            return json.loads(cleaned_value)
        except json.JSONDecodeError:
            return None

def load_existing_data(filename):
    """ Loads JSON data from a file if it exists, otherwise returns an empty dictionary. """
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                print("Error reading the JSON file. Starting with an empty dictionary.")
                return {}
    return {}

def save_json_data(data, filename):
    """ Saves the dictionary to a JSON file with pretty formatting. """
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

# Load existing data or initialize an empty dictionary
data_words = load_existing_data('data_words.json')

# Read words from file
with open('1212.txt', 'r') as f:
    words = f.readlines()

# Process each word and update the dictionary
for word in tqdm(words, desc="Processing words", unit="word"):
    cleaned_word = word.strip()
    if cleaned_word not in data_words:
        word_family_etymology = convert_str_to_json(get_word_family_etymology(cleaned_word))
        if word_family_etymology:
            data_words[cleaned_word] = word_family_etymology
            save_json_data(data_words, 'data_words.json')  # Save progress periodically
        else:
            time.sleep(5) 
            word_family_etymology = convert_str_to_json(get_word_family_etymology(cleaned_word))
            if word_family_etymology:
                data_words[cleaned_word] = word_family_etymology
                save_json_data(data_words, 'data_words.json')  # Save progress periodically

        time.sleep(2)
# Save the final state after all processing is complete
save_json_data(data_words, 'data_words.json')


In [ ]:
def generate_html(word, word_family):
    html_content = f"<p>Word Family of <strong>{word}</strong>:</p>\n<ul>\n"
    for item in word_family:
        html_content += f"  <li><strong>{item['word_form']}</strong> ({item['part_of_speech']}) - {item['definition']}</li>\n"
    html_content += "</ul>"
    return html_content

for key, value in data_words.items():
    data_words[key]['word_family'] = generate_html(word, data_words[key]['word_family'])
    